# <center>Digital Image Processing - Problem Set 4</center>

### Student Names:

### Instructions

This Problem Set covers the topics of Morphological operations, Region filling, Canny edge detection, Hough Transform, Thresholding, Watershed segmentation, $k$-means segmentation.<br>

Your solutions to the following problems should include commented source code and a short description of each function. You should test your functions with several input images, besides the ones provided here. Include the input and output images that you used for experimentation. Analyze your results. If you discover something interesting, let us know!

## Problem 1

Write a function that performs the following sequence of morphological operations on a binary image and displays the intermediate and final results.

1. Create a square-shaped structuring element of size 3 by 3.
2. Erode the input image.
3. Dilate the result of b.
4. Dilate the result of c.
5. Erode the result of d.

Write a second function that performs the following operations and displays the intermediate and final results.

1. Create a square-shaped structuring element of size 3 by 3.
2. Perform opening of the input image.
3. Compute the closing of the result from b.

Apply both functions to the image <tt>noisy_fingerprint.jpg</tt>. Discuss the differences and similarities between the final results of using both functions.


## Problem 2

Write a function that implements the region filling function discussed in class.
In this function you will need to use an interactive plot, so we need to disable <tt>inline</tt> plotting. This can be done with the following code:


The algorithm should proceed as follows:

1. Open and display the input binary image.
2. Using the <tt>plt.ginput</tt> function, specify a pixel within the region that you want to fill. Record the $(x, y)$ coordinate of the selected pixel.
3. Generate $X_0$, a binary image whose size is equal to the size of the input image. The pixel at the coordinate selected in the previous step should be set to 1, all other pixels should be set to zero.
4. Compute $X_k$ according to the following equation. Repeat until convergence, i.e. $X_k = X_{k-1}$
\begin{equation}
X_k=(X_{k-1} ⊕ B) ∩ A^c
\end{equation}

5. Compute the union between the final $X_k$ and the original image to fill the region.

Apply your function to the image <tt>spheres.jpg</tt> to fill in all the black circles.

In [3]:
import cv2
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import sys
from Tkinter import *

def reg_filling(src):
    fig = plt.figure();
    plt.imshow(src,cmap='gray')
    plt.title('Select coordinate')
    coord= plt.ginput(1)  #Specify coordinate in image
    xcoord= int(round(coord[0][0]));ycoord=int(round(coord[0][1])) #Separate both x and y coordinates
    X = np.zeros((src.shape[0], src.shape[1]),dtype=np.uint8) #This is X0
    X[ycoord,xcoord]=255 #Replace matrix with coordinates given in plt.ginput
    B = 255*cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3)).astype(np.uint8)  #Define structuring element 
    A = src  #A is the image
    A_C= cv2.bitwise_not(A)  #Complement
    #Start operation
    x_prev = X[:,:]  #Copy of X (X(k-1))
    X = cv2.dilate(X,B,iterations=1)
    X = np.bitwise_and(X,A_C)  #Xk
    cont = 0
    while(not(np.array_equal(x_prev,X))):  #Loop until convergence (In this case x_final = X)
        x_prev = X[:,:]
        X = cv2.dilate(X,B,iterations =1)
        X = np.bitwise_and(X,A_C)
        cont +=1
    plt.close(fig)
    fig1 = plt.figure()
    plt.imshow(X,cmap='gray')
    plt.show()
    imgsal = np.bitwise_or(X,A)
    plt.close(fig1)
    return imgsal


def exit():
    root.destroy()
#Tk interface to specify number of holes to be filled
root = Tk()
root.minsize(width=300, height=200)
r = Label (root, text= "¿Cuántas veces desea rellenar?")
r.place(relx=0.5, rely=0.25, anchor=CENTER)
r1 = Label (root, text= "Recommended: 17")
r1.place(relx=0.5, rely=0.375, anchor=CENTER)
text_entered = StringVar()
e = Entry(root, textvariable=text_entered)
e.place(relx=0.5, rely=0.5, anchor=CENTER)

button = Button(root,text="Close",command=exit)
button.place(relx=0.5, rely=0.75, anchor=CENTER)

root.mainloop()
src = cv2.imread('spheres.jpg',0)
global text_entered
content = text_entered.get()
filled_img = reg_filling(src)

    
for i in range(int(content)-1):
    filled_img = reg_filling(filled_img)
orig = plt.figure();    
plt.imshow(src,cmap='gray')
plt.title('Original image')
plt.figure();
plt.imshow(filled_img,cmap='gray')
plt.title('Final image with holes filled') 





<h3>- Descripción de la función desarrollada:</h3>    
<p style="text-align:justify">La función reg_filling desarrollada en este inciso recibe como parámetro la imagen a la cual se le desea rellenar los vacíos o huecos y retorna una imagen con los vacíos o huecos rellenados. El número de vacíos a rellenar es especificado a través de una interfaz de la libreria Tkinter.</p>

<h3>- Descripción de las funciones de librerías utilizadas:</h3>
<p style="text-align:justify">Para el desarrollo de este inciso se utilizaron las librerias OPENCV, Matplotlib y Numpy a través de las cuales se emplearon distintas funciones de las que se puede resaltar ginput() mediante la cual es posible capturar las coordenadas de una figura en las cuales se presiona click. El próposito de utilizar esta función es en este caso el de especificar la coordenada del vacío o hueco que se desea rellenar. Esta función recibe como parámetro de entrada el número de clicks que se van a grabar por lo que en este caso se especificó el número 1 debido a que solo se registrará una coordenada. Cabe destacar que para utilizar esta función fue necesario desactivar el modo "inline plotting" de la librería matplotlib y se decidió utilizar Qt debido a que ya se contaba con esta librería adicional a su facilidad de uso. Adicionalmente, se utilizó la función getStructuringElement() de la librería OpenCV la cual recibe como parámetro de entrada la forma del elemento estructurante a definir. En este caso se definió un elemento estructurante en forma de cruz y de tamaño 3x3. Para definir la forma de cruz de este elemento estructurante se utilizó la funcion MORPH_CROSS de la librería openCV.</p>



## Problem 3

Connected component labeling (also known as connected component analysis, blob extraction, region labeling, blob discovery, or region extraction) is an algorithm that uniquely labels connected components. The definition of connected components depends on the type of pixel adjacency used (pixel adjacency is defined and described in the book). In this problem, we will look at how the result of a connected component labeling algorithm changes when using 4-adjacency or 8-adjacency.

Write a function, <tt>labelConnectedComponents</tt>, which identifies and labels each connected component in a binary image. The function should take as an input the kind of adjacency to be used (either 4-adjacency or 8-adjacency). The function should also display the resulting connected components as shown below (particular coloring may differ). Apply this function to the images crosses.gif and chessboard.gif. Compare the connected components obtained when using 4-adjacency against those obtained when using 8-adjacency. Please comment your results: are these what you expected?

<b>Hint</b>: Use the functions <tt>cv2.findContours</tt> and <tt>cv2.drawContours</tt>. The following Wikipedia article is also a useful reference:
http://en.wikipedia.org/wiki/Connected_Component_Labeling

<img src="files/crosses.png"/>
<img src="files/connected.jpg"/>

## Problem 4

The blog ‘Steve on Image Processing’ contains useful information about image processing with MATLAB. One of Steve’s posts talks about using morphological operations to identify characters containing a tall vertical segment. The post is at:
http://blogs.mathworks.com/steve/2008/07/14/opening-by-reconstruction/

Replicate the results with Python and OpenCV. You may find that the functions <tt>cv2.findContours</tt> and <tt>cv2.floodFill</tt> are useful for this.

## Problem 5 

Write a function that detects edges in an image using Canny’s method, which is available in the function <tt>cv2.Canny</tt>.
Your function should first smooth the image using a Guassian filter, which you can do with <tt>cv2.GaussianBlur</tt>.
Try out Canny's method with different sizes the smoothing kernel and analyze the results.
Also, try changing the thresholds in the edge detection process. What are the effects of each parameter?

## Problem 6

Write a function that uses the Hough Transform algorithm for line detection. You can use the implementation available in OpenCV <tt>cv2.HoughLines</tt> in your function. Note that this function returns lines in the $(\rho, \theta)$ space, so you need to perform the appropiate conversions before plotting them.

Run the line detection function on an edge map obtained using Canny's method. Display the detected lines on top of the input image. You should use several images for your tests, including <tt>building.jpg</tt>.

## Problem 7

Write a function that implements a simple adaptive thresholding algorithm as described below:

1. Select an initial estimate for $T$. A good initial value for $T$ would be equal to the average of the maximum and minimum intensity value of the image.
2. Threshold the image using $T$. This produces two groups: $G_1$, with pixels whose intensity is equal or less than $T$; and $G_2$, with pixels whose intensity is greater than $T$.
3. Compute the intensity averages $\mu_1$ and $\mu_2$ for the pixels in regions $G_1$ and $G_2$.
4. Compute a new threshold value
\begin{equation}
T=\frac{\mu_1 + \mu_2}{2}
\end{equation}

5. Repeat steps 2-4 until the difference in $T$ between successive iterations is less than a predefined parameter $T_0$.

Apply your function to the image <tt>fingerprint.jpg</tt>. Analyze your results.


## Problem 8

Write a function that uses the $k$-means algorithm to perform image segmentation using RGB pixel colors as features.
Write also a variant of this segmentation approach that uses the H and S components of each pixel represented in the HSI color space. Here, you should use normalized H and S values, so that both variables range between 0 and 1.
Apply your functions to the image <tt>mms.jpg</tt>. Select an appropriate value of $k$.
